In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def  DataPrep(location, thres):
    df_outages = pd.read_csv('../Data/PowerOutageData/OutagesComplete.csv')
    df_outages = df_outages.rename(columns = {"RecordDate" : "datetime"})
    df_outages['datetime'] = pd.to_datetime(df_outages['datetime'])

    
    util = location[0]
    countyName = location[1]
    cityName = location[2]

    df_out = df_outages.loc[(df_outages["UtilityName"] == util) & (df_outages["CountyName"] == countyName) & (df_outages["CityName"] == cityName)]
    df_out['outageValue'] = 100.0 * df_out["CustomerHoursOutTotal"]/(df_out["CustomerHoursTrackedTotal"])
    
    #df_out.loc[df_out['outageValue'] < thres, 'outageValue'] = 0
    #df_out.loc[df_out['outageValue'] >= thres, 'outageValue'] = 1
    
    average = df_out['outageValue'].mean()
    maximum = df_out['outageValue'].max()
    minimum = df_out['outageValue'].min()
    zeroCount = len(df_out[df_out['outageValue'] == 0])
    zeroP = zeroCount/len(df_out['outageValue']) * 100
    length = len(df_out['outageValue'])
    df_out.hist(column='outageValue',bins=[0, 100, 200, 300, 400, 500, 600 ,700, 800, 900, 1000, 2000, 5000, 100000])
    plt.title('Histogram of Column Name')
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.savefig("../Results/StatisticsResults/" + location[2] + '_histogram_20230903.png')
    plt.show()
    
    f = open("../Results/StatisticsResults/"+location[2]+"_Statistics_20230903.txt", "w")
    print("average: " + str(average), file = f)
    print("max: " + str(maximum), file = f)
    print("min: " + str(minimum), file = f)
    print("Zeroes: " + str(zeroCount), file = f)
    print("Zero percent: " + str(zeroP), file = f)
    print("Length: " + str(length), file = f)        
    f.close()
    
    df_weather = pd.read_csv("../Data/WeatherData/"+cityName+"_weather.csv")
    df_weather = df_weather[['datetime', 'temp', 'humidity', 'precip', 'windspeed', 'solarradiation']]
    df_weather['datetime'] = pd.to_datetime(df_weather['datetime'])
    df_weather['Season'] = ((df_weather['datetime'].dt.month).floordiv(3)).mod(4)
    
    df = df_weather.merge(df_out, how = 'left', on = 'datetime')
    
    df = df.dropna(axis = 0, how = 'any')
    
    df.to_csv("../Data/GeneratedData/"+cityName+"_Weather_Outages.csv", index= False)
    
    return (average, maximum, minimum, zeroP)
    
print("done")


In [ ]:
locationList = [
    ["Pacific Gas and Electric Company", "Fresno", "Fresno"], 
                ["Pacific Gas and Electric Company", "San Francisco", "San Francisco"], 
                ["Pacific Gas and Electric Company", "Santa Clara", "San Jose"],
                ["San Diego Gas & Electric", "San Diego", "San Diego"],
                ["Los Angeles Department of Water & Power", "Los Angeles", "Los Angeles"], 
                ["Puget Sound Energy", "Pierce", "Tacoma"], 
                ["Puget Sound Energy", "King", "Bellevue"], 
                ["Puget Sound Energy", "King", "Seattle"],
                ["Clark County PUD", "Clark", "Vancouver"],
                ["Eugene Water & Electric Board", "Lane", "Eugene"]
                ]

In [ ]:
df_res = pd.DataFrame(columns= ["location", "average", "max", "min", "ZeroPercent"])
for location in locationList:
    avg, mx, mn, zP = DataPrep(location, 0.1)
    new_row = {'location': location[2], 'average':avg, 'max':mx, 'min':mn, 'ZeroPercent':zP}
    df_res = df_res.append(new_row, ignore_index=True) 
print (df_res.sort_values(by = ["ZeroPercent"]))


In [ ]:
import plotly.figure_factory as ff

df_res.to_csv("../Results/StatisticsResults/StatisticsSummary.csv", index = False)
df_res = df_res.round(2)

fig = ff.create_table(df_res)
fig.update_layout(
autosize=False,
width=500,
height=200,
)

fig.write_image("../Results/StatisticsResults/StatTable.png", scale=2)